In [ ]:
import shelve
import warnings
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

import numpy as np
import scipy.stats
from skimage.io import imread

import PaperPlotting
import EpiPaperPlotting

from StatisticTesting import stats_file, log_stat_to_file
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'invalid value encountered in')

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.sans-serif']='Arial'
plt.rcParams['figure.constrained_layout.use']=True

figure_path = 'Figures'
two_photon_db = PaperPlotting.two_photon_db
epi_db = EpiPaperPlotting.epi_db
open(stats_file, 'w').close()

# Figure 1 : Cellular Properties

In [ ]:
log_stat_to_file('Figure 1')
fig = plt.figure(figsize=(8,6))
gs = GridSpec(nrows = 4, ncols = 12, figure=fig)

example_animals = (('F2330', 16, 'Naive'), 
                   ('F2312', 29, 'Brief'), 
                   ('F2397', 18, 'Extended'))
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'invalid value encountered in')
    with shelve.open(two_photon_db) as db:
        for idx, (aid, cell_num, title) in enumerate(example_animals): 
            ax = fig.add_subplot(gs[idx, 0:-4])
            PaperPlotting.plot_fluorescence_responses(db[aid], cell_num,ax, avgtype='median')
            ax.set_ylim((-1, ax.get_ylim()[1]))
            ax.set_yticks(range(0,5, 2))
            ax.set_xticks(range(0,10*17, 10))
            ax.set_xticklabels([])
            eo = db[aid].get_expt_parameter('eo')
            age = db[aid].get_expt_parameter('age')
            ax.set_title(f'{title}: p{age} EO+{eo}')
            
            ax = fig.add_subplot(gs[idx,-4:])
            PaperPlotting.plot_tuning_curve(db[aid], cell_num, ax)
            rsq= db[aid].binoc.analysis['Full'].metrics['dr_rsq'][cell_num]
            osi = db[aid].binoc.analysis['Full'].metrics['OSI'][cell_num]
            ax.text(270, 1.75, f'$R^2 = $ {rsq:0.2f}\nOSI = {osi:0.2f}')
            ax.set_ylim((0,2))
            ax.set_ylabel('$\Delta F/F$')
            ax.set_xlabel('Direction ($^\circ$)')
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
        ax = fig.add_subplot(gs[3, -3:])
        responsive = PaperPlotting.plot_percent_responsive(db,['Naive', 'Brief', 'Extended'], 'dlx', ax)

        for idx, (metric, rng) in enumerate(zip(['OSI', 'DSI', 'variability',], [(0,1),(0,1),(0,1)])):
            ax = fig.add_subplot(gs[3,idx*3:idx*3+3])

            PaperPlotting.cumulative_plot(db, 
                                          metric, 
                                          ax, 
                                          virus='dlx', 
                                          value_range=rng, 
                                          eo_groups=['Naive', 'Brief', 'Extended'],
                                          rsq_threshold=0.6)
            ax.legend(borderpad=0, frameon=False, shadow=False, fontsize='x-small')


fig.savefig(fr'{figure_path}\Figure_1.pdf')

## Figure 1- Supplement 1

In [ ]:
fig = plt.figure(figsize=(12,3))
cohens_dict = {}
with shelve.open(two_photon_db) as db:
    for ax_idx, eo in enumerate(['Naive-dlx', 'Brief-dlx', 'Extended-dlx']):
        ax = fig.add_subplot(1,4,ax_idx+2)
        tuned_osi = np.empty((0))
        tuned_cohens = np.empty((0))
        untuned_osi = np.empty((0))
        untuned_cohens = np.empty((0))
        for aid in PaperPlotting.expt_groups[eo]:
            expt = db[aid]
            orth, blank, _ = PaperPlotting.compare_cohens(expt)
            osi = expt.binoc.analysis['Full'].metrics['OSI'].values
            untuned = expt.binoc.analysis['Full'].responsive() & ~expt.binoc.analysis['Full'].metrics['OSI_sig'].values
            tuned = expt.binoc.analysis['Full'].responsive() & expt.binoc.analysis['Full'].metrics['OSI_sig'].values
            tuned_osi = np.concatenate((tuned_osi, osi[tuned]))
            tuned_cohens = np.concatenate((tuned_cohens, blank[tuned]))
            untuned_cohens = np.concatenate((untuned_cohens, blank[untuned]))
            ax.scatter(blank[tuned], osi[tuned],  marker='.', color='k', zorder=3)
        valid_cells = np.isfinite(tuned_cohens) & np.isfinite(tuned_osi)
        r, p = scipy.stats.pearsonr(tuned_cohens[valid_cells], tuned_osi[valid_cells])
        cohens_dict[eo+'-tuned'] = tuned_cohens[valid_cells]
        cohens_dict[eo+'-untuned'] = untuned_cohens[np.isfinite(untuned_cohens)]
        
        
        ax.set_title(f'{eo.split("-")[0]}')
        ax.set_xlabel('SAI')
        ax.set_ylabel('OSI')
        ax.set_xlim([-2,6])
        ax.set_ylim([0,1.1])
        ax.text(3, .85, f'r = {r:0.2f}', color='k')
        log_stat_to_file(f'{eo}: r={r:0.2f} (p={p:0.4f})')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        
ax = fig.add_subplot(141)
PaperPlotting.violin_plot_cohens(cohens_dict, ax)

fig.tight_layout()
fig.savefig(fr'{figure_path}\Supplementary_figure_1.pdf')

## Supplementary Figure 2

In [ ]:
log_stat_to_file('Supplmentary Figure 2')
fig = plt.figure(figsize=(8,4.5))
gs = GridSpec(nrows = 3, ncols = 12, figure=fig)

example_animals = (('F2330', 16, 'Naive'), 
                   ('F2312', 29, 'Brief'), 
                   ('F2397', 18, 'Extended'))
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'invalid value encountered in')
    with shelve.open(two_photon_db) as db:        
        for e_idx, eye in enumerate(['contra', 'ipsi']):
            ax = fig.add_subplot(gs[e_idx, -3:])
            responsive = PaperPlotting.plot_percent_responsive(db,['Naive', 'Brief', 'Extended'], 'dlx', ax, eye=eye)

            for idx, (metric, rng) in enumerate(zip(['OSI', 'DSI', 'variability',], [(0,1),(0,1),(0,1)])):
                ax = fig.add_subplot(gs[e_idx,idx*3:idx*3+3])
                PaperPlotting.cumulative_plot(db, 
                                            metric, 
                                            ax, 
                                            virus='dlx',
                                            eye=eye, 
                                            value_range=rng, 
                                            eo_groups=['Naive', 'Brief', 'Extended'],
                                            rsq_threshold=0.6)
                ax.legend(borderpad=0, frameon=False, shadow=False, fontsize='x-small')
        ax = fig.add_subplot(gs[2,:3])
        PaperPlotting.binocular_cumulative_plot(db, 'monocularity', ax=ax, eo_groups=['Naive', 'Brief', 'Extended'], virus = 'dlx', value_range=(0,1), osi_threshold=0, rsq=0)

        for m_idx, metric in enumerate(['|binoc-ipsi|', '|binoc-contra|', '|ipsi-contra|']):
            ax = fig.add_subplot(gs[2, (m_idx+1)*3:(m_idx+2)*3])
            PaperPlotting.binocular_cumulative_plot(db, metric, ax=ax, eo_groups=['Brief', 'Extended'], virus = 'dlx', value_range=(0,45))
            ax.set_xlim(0,45)
            ax.set_xticks(range(0,46,15))

fig.savefig(fr'{figure_path}\Supplementary_figure_2.pdf')

# Figure 2 Network encoding

In [ ]:
fig = plt.figure(figsize=(8,4))
gs = GridSpec(nrows=2, ncols=4, figure = fig)

example_animals = (('F2330', 'Naive'),
                   ('F2312',  'Brief'),
                   ('F2397',  'Extended'))
corr_type='total'
eo_list = ['Naive', 'Brief', 'Extended']
virus = 'dlx'
with shelve.open(two_photon_db) as db:
    
    for idx, (aid, title) in enumerate(example_animals):
        if aid not in db:
            continue
        expt = db[aid]
        age = expt.get_expt_parameter('age')
        eo = expt.get_expt_parameter('eo')
        ax = fig.add_subplot(gs[0, idx])
        expt.binoc.analysis['Full'].plot_trial_to_trial_correlation_matrix(ax=ax, colorbar=False)
 
        
    PaperPlotting.plot_trial_variability_metric(db, 
                                            eo_list,
                                            virus,
                                            ax=fig.add_subplot(gs[0, 3]), 
                                            plot_orth=True)
    
    
    ax_list = []
    for i in range(len(eo_list)):
        ax_list.append(fig.add_subplot(gs[1, i]))
            
    _,_ =PaperPlotting.plot_template_decoding(db,
                                                eo_list,
                                                ax_list)
    
    ax = fig.add_subplot(gs[1,3])
    derivatives = PaperPlotting.plot_decoding_derivative(db, eo_list, ax)
    
    
fig.savefig(f'{figure_path}/Figure_2.pdf', dpi=1200)

# Figure 3 - Activity Patterns

In [ ]:
log_stat_to_file('Figure 3')
example_animals ={'F2334': ((270,40), (40,290)),
                  'F2312': ((270,40), (0,250)),
                  'F2395': ((245,15), (25,275)),
                  }
eo_list = ['Naive', 'Brief', 'Extended']
virus='dlx'
fig = plt.figure(figsize=(3*4,3*5))
ax_list=[]
gs = GridSpec(ncols=10, nrows=5, figure=fig)
for idx in range(len(example_animals)):
    animal_axes=[]
    for i in range(4):
        animal_axes.append(fig.add_subplot(gs[i, idx*2+3:(idx+1)*2+3]))
    ax_list.append(animal_axes)
    
with shelve.open(epi_db) as db:
    for ax, (aid, fov) in zip(ax_list, example_animals.items()):
        if aid not in db:
            continue 
        expt = db[aid]
        expt.analysis['Full'].plot_correlation_map(ax[3], fov=fov, scalebar=1000)
        EpiPaperPlotting.plot_difference_map(expt, 
                                            ax, 
                                            fov=fov, 
                                            override=False,
                                            scalebar=1000)
    
    ax = fig.add_subplot(gs[2,:3])
    EpiPaperPlotting.fraction_of_tuned_pixels(db, ax, virus=virus, eo_list=eo_list)
    
    ax = fig.add_subplot(gs[3,:3])
    epi_binned_data= EpiPaperPlotting.plot_binned_correlations(db, ax, virus=virus, eo_list=eo_list, field='Full', corr_type='total')
    ax.set_xlim(0, 1.6)
    ax.set_xticks([0,.5,1,1.5])
    ax.set_ylabel('')
    
    
two_photon_example_animals = (('F2334', 15, 'Naive'),
                   ('F2312', 29, 'Experienced'),
                   ('F2397', 18, 'Mature'))

with shelve.open(two_photon_db) as db:
    for idx, (aid, seed_cell, title) in enumerate(two_photon_example_animals):
            if aid not in db:
                continue
            expt = db[aid]
            age = expt.get_expt_parameter('age')
            eo = expt.get_expt_parameter('eo')
            ax = fig.add_subplot(gs[4,idx*2:(idx+1)*2])
            
            ax.set_title(f'{title}\n p{age} EO+{eo}')
            expt.binoc.analysis['Full'].scatter_correlation(ax=ax, seed_cell=seed_cell, corrtype='total')

    ax = fig.add_subplot(gs[4,-3:])
    PaperPlotting.bar_plot_correlation_clustering(db,
                                                ['Naive', 'Brief', 'Extended'],
                                                'dlx',
                                                ax=ax,
                                                bin_range=((0, 100), (300, 400)),
                                                nbins=1,
                                                corrtype='total', grouped=False, stat_test=True)

fig.savefig(f'{figure_path}/Figure_3.pdf', dpi=1200)

## Supplementary Figure 2

In [ ]:
fig = plt.figure(figsize=(6,4))
gs = GridSpec(nrows = 5, ncols = 16, figure = fig)

example_animals ={'F2334': (np.arange(1,10,2), 5, ((270,40), (40,290))),
                  'F2312': (np.arange(1,10,2), 0, ((270,40), (0,250))),
                  'F2395': (np.arange(1,10,2), 3, ((245,15), (25,275))),
                  }


with shelve.open(epi_db) as db:
    for a_idx, (animal_id, (trials,stim, boundaries)) in enumerate(example_animals.items()):
        ax_list = [fig.add_subplot(gs[a_idx,3*ax_idx+1:3*(ax_idx+1)+1]) for ax_idx in range(5)]
        EpiPaperPlotting.plot_single_trial_response(db[animal_id],
                                                    ax_list,
                                                    trials,
                                                    stim,
                                                    boundaries)
    
    ax = fig.add_subplot(gs[3:,:7])
    wavelengths = EpiPaperPlotting.wavelength_plot(db, ax, eo_list=['Naive', 'Brief', 'Extended'])

    ax1 = fig.add_subplot(gs[3:,7:11])
    ax2 = fig.add_subplot(gs[3:,11:])
    EpiPaperPlotting.cropped_roi(db, ax1, ax2)
    for ax in [ax1, ax2]:
        for position in ['top', 'right', 'bottom', 'left']:
            ax.spines[position].set_visible(False)
     
fig.savefig(fr'{figure_path}/Supplementary_figure_3.pdf', dpi=1200)

## Supplementary Figure 4

In [ ]:
fig = plt.figure(figsize=(6,4), constrained_layout=True)
gs = GridSpec(nrows = 4, ncols =21, figure=fig)
example_animals = [('Naive', 'F2334',12, [0,1,2,3,4]),
                    ('Brief', 'F2312',0,[2,3,4,5,6,]),
                    ('Extended', 'F2397',2,[0,1,2,3,4])]
with shelve.open(two_photon_db) as db:
    for a_idx, (label, example_animal, stim, trials) in enumerate(example_animals):
        
        ax = fig.add_subplot(gs[a_idx, 0:2])
        ax.imshow(imread(f'TifFiles/{example_animal}.tif'), cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        expt = db[example_animal]
        responses = expt.binoc.analysis['Full'].single_trial_responses()
        responsive = expt.binoc.analysis['Full'].responsive()
        positions = expt.binoc.roi_positions()[responsive]
        stim_responses=responses[:,stim,trials]
        stim_responses=stim_responses/ np.repeat(np.expand_dims(np.max(responses[:,:-1,:], axis=(1,2)),axis=1),stim_responses.shape[1],axis=1)
        stim_responses=stim_responses[responsive]
        for trial_idx in range(len(trials)):
            ax = fig.add_subplot(gs[a_idx,(trial_idx+1)*2:(trial_idx+2)*2])
            PaperPlotting.plot_single_trials(positions, stim_responses[:,trial_idx], ax, s=5)
            ax.set_aspect('equal')
        ax = fig.add_subplot(gs[a_idx, -7:])
        PaperPlotting.plot_raster(expt, ax, label)

fig.savefig(fr'{figure_path}/Supplementary_figure_4.pdf', dpi=1200)

# Figure 4 - Visual Experience

In [ ]:
log_stat_to_file('Figure 4')
fig = plt.figure(figsize=(8,6))
gs = GridSpec(nrows = 3, ncols = 12, figure=fig)

example_animals = (('F2361', 33,'Binocular Deprivation'), 
                   ('F2371', 4,'Recovery'), 
                   )
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'invalid value encountered in')
    with shelve.open(two_photon_db) as db:
        for idx, (aid, cell_num, title) in enumerate(example_animals):  
            age = db[aid].binoc.metadata.expt['age']
            eo = db[aid].binoc.metadata.expt['eo']
            ax = fig.add_subplot(gs[idx, 0:-4])
            PaperPlotting.plot_fluorescence_responses(db[aid], 
                                                      cell_num,
                                                      ax,
                                                      avgtype='median')
            ax.set_ylim((-1, ax.get_ylim()[1]))
            ax.set_yticks(range(0,5, 2))
            ax.set_title(f'{title}: p{age} EO+{eo}')
            ax.set_xticks(range(0,10*17, 10))
            ax.set_xticklabels([])

            ax = fig.add_subplot(gs[idx,-4:])
            PaperPlotting.plot_tuning_curve(db[aid],
                                            cell_num, 
                                            ax)
            rsq= db[aid].binoc.analysis['Full'].metrics['dr_rsq'][cell_num]
            osi = db[aid].binoc.analysis['Full'].metrics['OSI'][cell_num]
            dsi = db[aid].binoc.analysis['Full'].metrics['DSI'][cell_num]
            ax.text(270, 1.75, f'$R^2 = $ {rsq:0.2f}\nOSI = {osi:0.2f}\nDSI = {dsi:0.2f}')

            ax.set_ylim((0,2))
            ax.set_ylabel('$\Delta F/F$')
            ax.set_xlabel('Orientation ($^\circ$)')
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)

        ax = fig.add_subplot(gs[2, -3:])
        PaperPlotting.plot_percent_responsive(db,['BD', 'Recovery'], 'dlx', ax)

        for idx, (metric, rng) in enumerate(zip(['OSI','DSI', 'variability',], [(0,1),(0,1),(0,1)])):
            ax = fig.add_subplot(gs[2,idx*3:idx*3+3])
            PaperPlotting.cumulative_plot(db, 
                                          metric, 
                                          ax, 
                                          virus='dlx', 
                                          value_range=rng, 
                                          eo_groups=['BD', 'Recovery', 'Brief'],
                                          rsq_threshold=0.6,
                                          skip_sem = ['Brief'])
            ax.legend(borderpad=0, frameon=False, shadow=False, fontsize='x-small')

fig.savefig(fr'{figure_path}/Figure_4.pdf')

# Figure 5 - Experience Network Response

In [ ]:
log_stat_to_file('Figure 5')
fig = plt.figure(figsize=(8, 8))
gs = GridSpec(nrows=3, ncols=12, figure=fig)
example_animals = (('F2376', 15, 'BD'),
                ('F2371', 4, 'Recovery'),
                )
corr_type='total'
eo_list = ['BD', 'Recovery']
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', 'invalid value encountered in')
    with shelve.open(two_photon_db) as db:
        for idx, (aid, seed_cell, title) in enumerate(example_animals):
            if aid not in db:
                continue
            expt = db[aid]
            age = expt.get_expt_parameter('age')
            eo = expt.get_expt_parameter('eo')
           
            ax = fig.add_subplot(gs[0, idx*3:(idx+1)*3])
            ax.set_title(f'{title}\n p{age} EO+{eo}')
            expt.binoc.analysis['Full'].scatter_correlation(ax=ax, seed_cell=seed_cell, corrtype=corr_type)

            if idx == len(example_animals)-1:
                cbar = plt.colorbar(ScalarMappable(
                    norm=Normalize(-1, 1), cmap='bwr'), ax=ax, ticks=[-1, 0, 1])
                cbar.ax.set_ylabel('Response Correlation', rotation=270)
            ax = fig.add_subplot(gs[1, idx*3:(idx+1)*3])
            ax = fig.add_subplot(gs[1, idx*3:(idx+1)*3])
            expt.binoc.analysis['Full'].plot_trial_to_trial_correlation_matrix(ax=ax)
        

        for v_idx, virus in enumerate(['dlx']):
            ax = fig.add_subplot(gs[0, -3:])
            x = PaperPlotting.bar_plot_correlation_clustering(db,
                                                              eo_list,
                                                              virus,
                                                              ax=ax,
                                                              bin_range=(
                                                                  (0, 100), (300, 400)),
                                                              nbins=1,
                                                              corrtype=corr_type, grouped=False, stat_test=True
                                                              )

            ax.set_xticklabels(['BD', 'R'])


            ax = fig.add_subplot(gs[1, -3:])
            PaperPlotting.plot_trial_variability_metric(db, 
                                                        eo_list,
                                                        virus,
                                                        ax=ax)
            ax.set_xticklabels(['BD', 'R'])
            ax_list = []
            for i in range(len(eo_list)):
                ax_list.append(fig.add_subplot(gs[2, i*4:(i+1)*4]))
            
            PaperPlotting.plot_template_decoding(db,
                                                 eo_list,
                                                 ax_list)
            ax = fig.add_subplot(gs[-4:])
            PaperPlotting.plot_decoding_derivative(db, eo_list, ax)

    fig.savefig(fr'{figure_path}/Figure_5.pdf')


## Supplementary Figure 5

In [ ]:

fig = plt.figure(figsize=(8,6))
gs = GridSpec(nrows = 4, ncols = 21, figure = fig)
example_animals = [('BD', 'F2376', 14, [0,3,6,8,9]),
('Recovery', 'F2371',14, [1,3,4,7,8])]

with shelve.open(two_photon_db) as db:
    for a_idx, (label, example_animal, stim, trials) in enumerate(example_animals):
        
        ax = fig.add_subplot(gs[a_idx+2, 0:2])
        ax.imshow(imread(f'TifFiles/{example_animal}.tif'), cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        expt = db[example_animal]
        responses = expt.binoc.analysis['Full'].single_trial_responses()
        positions = expt.binoc.roi_positions()
        stim_responses=responses[:,stim,trials]
        stim_responses=stim_responses/ np.repeat(np.expand_dims(np.max(responses[:,:-1,:], axis=(1,2)),axis=1),stim_responses.shape[1],axis=1)
        for trial_idx in range(len(trials)):
            ax = fig.add_subplot(gs[a_idx+2, (trial_idx+1)*2:(trial_idx+2)*2])
            PaperPlotting.plot_single_trials(positions, stim_responses[:,trial_idx], ax, s=5)
            ax.set_aspect('equal')
        ax = fig.add_subplot(gs[a_idx+2, -6:])
        PaperPlotting.plot_raster(expt, ax, label)
example_animals ={'F2374': (np.arange(0,10,2), 0, ((300,50), (-30,220))),
                  'F2372': (np.arange(1,10,2), 0, ((250,0), (50,300))),
                  }

with shelve.open(epi_db) as db:
    for a_idx, (animal_id, (trials,stim, boundaries)) in enumerate(example_animals.items()):
        ax_list = [fig.add_subplot(gs[a_idx,3*ax_idx:(ax_idx+1)*3]) for ax_idx in range(5)]
        EpiPaperPlotting.plot_single_trial_response(db[animal_id],
                                                    ax_list,
                                                    trials,
                                                    stim,
                                                    boundaries)
    
        ax=fig.add_subplot(gs[a_idx, 15:18])
        db[animal_id].analysis['Full'].plot_correlation_map(ax, fov=boundaries, scalebar=1000, colorbar=False)
    
    ax1 = fig.add_subplot(gs[:2,-3:])
    wavelengths = EpiPaperPlotting.wavelength_plot(db, ax1, eo_list=['BD', 'Recovery'])

    
fig.savefig(fr'{figure_path}/Supplementary_figure_5.pdf', dpi=800)